In [1]:
import sys
sys.path.append("../../")
import libspn as spn
import numpy as np
import tensorflow as tf

In [2]:
num_decomps = 5
num_subsets = 5
num_input_mixtures = 5
num_roots = 4
num_mixtures = 5
input_dist = spn.DenseSPNGenerator.InputDist.MIXTURE

# Additive smoothing during learning
additive_smoothing=10
min_additive_smoothing=1
smoothing_decay=0.2

# Weight initialization
# weight_init_value = 1
weight_init_value = spn.ValueType.RANDOM_UNIFORM(1, 2)

# Type of inference during upward pass of learning
value_inference_type = spn.InferenceType.MARGINAL

In [3]:
data = []
with open('/home/jos/datasets/msbnc/msnbc990928.seq', 'r') as f:
    line = f.readline()
    while 'Sequences:' not in line:
        line = f.readline()
    f.readline()  # also skip blank line
    
    for line in f.readlines():
        seq = [int(a) - 1 for a in line.strip().split(' ')]
        if len(seq) <= 32:
            data.append([int(a) - 1 for a in line.strip().split(' ')])

print(len(data))
print(max([len(s) for s in data]))
print(max([max(s) for s in data]))
print(min([min(s) for s in data]))

980734
32
16
0


In [4]:
def sequence_lens(sequences):
    return [len(seq) for seq in sequences]

def pad_sequences(sequences, sequence_maxlen):
    ret = []
    for seq in sequences:
        arr = np.asarray(seq)
        ret.append(np.concatenate([
            -1 * np.ones((sequence_maxlen - len(seq),) + arr.shape[1:]),
            arr
        ]))
    return np.asarray(ret)



In [5]:
tf.reset_default_graph()
dense_gen = spn.DenseSPNGenerator(num_decomps=num_decomps, num_subsets=num_subsets, num_mixtures=num_mixtures, 
                                  input_dist=input_dist, 
                                  num_input_mixtures=num_input_mixtures)

ivs = spn.DynamicIVs(max_steps=32, num_vars=1, num_vals=17)

top_per_digit = []
# for d in range(10):
template_heads = dense_gen.generate(
    ivs, num_roots=num_roots, root_sum=False, root_name="DenseDigit")

interfaces = []
for i, head in enumerate(template_heads):
    intf = spn.DynamicInterface(name="Interface")
    intf.set_source(head)
    interfaces.append(intf)

interface_mixtures = []
for i, head in enumerate(template_heads):
    mixture = spn.Sum(*interfaces, name="InterfaceMixture")
    head.set_values(mixture, *head.values)

root = spn.Sum(*template_heads, name="Top")
spn.generate_weights(root, init_value=weight_init_value)
# latent = root.generate_ivs()

print(root.get_num_nodes())


34


In [6]:
sequence_len_ph = tf.placeholder(tf.int32, [None])
additive_smoothing_var = tf.Variable(additive_smoothing, dtype=spn.conf.dtype)
learning = spn.EMLearning(root, log=True, value_inference_type = value_inference_type,
                          additive_smoothing=additive_smoothing_var, sequence_lens=sequence_len_ph)
init_weights = spn.initialize_weights(root)
reset_accumulators = learning.reset_accumulators()
accumulate_updates = learning.accumulate_updates()
update_spn = learning.update_spn()
train_likelihood = learning.likelihood()
avg_train_likelihood = tf.reduce_mean(train_likelihood)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


[WARNING] [tensorflow:warning] From /home/jos/spn/libspn/libspn/utils/math.py:277: calling reduce_max (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


[WARNING] [tensorflow:warning] From /home/jos/spn/libspn/libspn/utils/math.py:281: calling reduce_sum (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
Use the `axis` argument instead


[WARNING] [tensorflow:warning] From /home/jos/spn/libspn/libspn/graph/sum.py:299: calling argmax (from tensorflow.python.ops.math_ops) with dimension is deprecated and will be removed in a future version.
Instructions for updating:
Use the `axis` argument instead


In [7]:
train_set = np.expand_dims(pad_sequences(data, 32), -1).transpose((1, 0, 2))
seq_lens = sequence_lens(data)

In [8]:
from tqdm import tqdm_notebook

In [9]:
sess = None

In [14]:

if sess is not None:
    sess.close()
sess = tf.Session()
    
sess.run(init_weights)
sess.run(reset_accumulators)

num_batches=84
batch_size = train_set.shape[1] // num_batches
prev_likelihood = 100
likelihood = 0
epoch = 0
while abs(prev_likelihood - likelihood)>0.025:
    prev_likelihood=likelihood
    likelihoods = []
    print("Epoch", epoch)
    pbar = tqdm_notebook(range(num_batches))
    for batch in pbar:
        start = (batch)*batch_size
        stop = (batch+1)*batch_size
#         print("EPOCH", epoch, "BATCH", batch, "SAMPLES", start, stop)
        # Adjust smoothing
        ads=max(np.exp(-epoch*smoothing_decay)*additive_smoothing, min_additive_smoothing)
        sess.run(additive_smoothing_var.assign(ads))
#         print("Smoothing: ", sess.run(additive_smoothing_var)) 
        # Run accumulate_updates
        train_likelihoods_arr, avg_train_likelihood_val, _, = \
                sess.run([train_likelihood, avg_train_likelihood, accumulate_updates],
                        feed_dict={ivs: train_set[:, start:stop],
                                   sequence_len_ph: seq_lens[start:stop]})
        # Print avg likelihood of this batch data on previous batch weights
        likelihoods.append(avg_train_likelihood_val)
        sess.run(update_spn)
        
        pbar.set_description("Likelihood: %s" % (avg_train_likelihood_val))
    likelihood = sum(likelihoods) / len(likelihoods)
    print("Avg likelihood: %s" % (likelihood))
    epoch+=1
    sess.run(reset_accumulators)
    

Epoch 0




Avg likelihood: -53.388345173427034
Epoch 1



Avg likelihood: -53.3055636542184
Epoch 2



Avg likelihood: -53.30555734180269
